# MGT 858: Database Systems
## Hospital Data Schema

The code written below seeks to add data to a postgresql database

### Import packages

In [ ]:
import random
import numpy as np
import pandas as pd
from datetime import timedelta
import datetime
import sqlite3
from faker import Faker

### Add randomly generated patients to the hospital 

In [ ]:
# Pre-existing SSNs for patients
SSN_list = []

# Create randomly generated data for new patients
def add_patients(num_to_add):
    fake = Faker()
    for i in range(num_to_add):
        SSN = 500000009 + i
        SSN_list.append(SSN)
        full_name = fake.name()
        address = fake.address()
        phone_number = str(random.randint(100,999)) + "-" + str(random.randint(1000,9999))
        insurance_id = random.randint(10000000,99999999)
        pcp = random.randint(1,13)
        with open("outputs/patients.txt", "a+") as patient_text:
            print(f"INSERT INTO patients (SSN, patient_name, address, phone, insurance_id, primary_care_provider) VALUES({SSN}, '{full_name}', '{address}', '{phone_number}', {insurance_id}, {pcp});", file=patient_text)  

In [ ]:
# Call add_patients function and add 100 generated patients
add_patients(500)

## Add patient stays to the hospital for given set of rooms

In [ ]:
# Import hospital length of stay data
stay_length_data = pd.read_csv("data/los_data.csv")

los_df = stay_length_data[['length_stay','prob_length_stay']]

days_in_hospital = los_df['length_stay'].tolist()
prob_in_hospital = los_df['prob_length_stay'].tolist()

days_in_hospital = np.array((days_in_hospital),dtype=np.float64)

In [ ]:
# Define hospital rooms
rooms =  [i+j for i in range(100, 700, 100) for j in range(1, 11)]

# Defines dates for which we will generate hospital stays
dates = pd.to_datetime(pd.date_range(start="2020-01-01",end="2020-08-01"))

# Defines a function that returns a dataframe of randomly generated patient stays
def patient_stays():
    # Create empty dataframe
    cols = ['date','patient_SSN','room_number','stay_start','stay_end']
    data = pd.DataFrame(columns = cols)
    
    # Create empty dictionaries to store patient and room data
    occupied_rooms = {}
    admitted_patients = {}
    
    # Separate week days by how likely patients will come to the hospital and stay
    busy_days = ['Monday','Thursday','Friday','Saturday']  
    
    # Iterate over all the days in the date range of interest
    for date in dates:
        
        if (date.strftime('%A') in busy_days):
            daily_arrival = random.randint(3,10) # Here, we say on a busy day, between 3-10 people will arrive randomly
        else:
            daily_arrival = random.randint(0,3) # Here, we say on a non-busy day, between 0-e people will arrive randomly
        
        daily_patient_arrival = random.sample(SSN_list,daily_arrival) # Select patients from the patient list
        
        # If the occupied_rooms/admitted_patients dictionary is not empty, subtract 1 day from the amount of 
        # time a patient is in the hospital from the total time they were set to stay in the hospital
        if len(occupied_rooms) != 0:
            occupied_rooms = {key: val - 1 for key, val in occupied_rooms.items()}
    
        if len(admitted_patients) != 0:
            admitted_patients = {key: val - 1 for key, val in admitted_patients.items()}
        
        # Delete patients and rooms from the list if they free up after a patient stay
        admitted_patients = {key:val for key, val in admitted_patients.items() if val >= 0}              
        occupied_rooms = {key:val for key, val in occupied_rooms.items() if val >= 0}    
        
        # Iterate over the randomly generated list of patients
        for patient in daily_patient_arrival:
            
            # If the randomly generated patient is already in the hospital, do nothing; 
            # If the randomly generated patient is new, assign them a room and hospital stay length in days
            if (patient not in admitted_patients):
                
                # Randomly sample a room
                assigned_room = random.choice(rooms)
                
                # If the room is already occupied, keep checking for a free room
                while (assigned_room in occupied_rooms): 
                    assigned_room = random.choice(rooms)

                # Here, we use the predetermined length of stay data     
                stay_length = np.random.choice(days_in_hospital,p=prob_in_hospital)
                stay_start = datetime.datetime.strptime(str(date),'%Y-%m-%d %H:%M:%S').date()
                stay_end = stay_start + datetime.timedelta(days=stay_length)
                
                # Add new patient/room (key) and stay_length (value) to the occupied_rooms and admitted_patients dictionaries
                occupied_rooms[assigned_room] = stay_length
                admitted_patients[patient] = stay_length

                data = data.append({'date':stay_start,'patient_SSN':patient,'room_number':assigned_room,'stay_start':stay_start,'stay_end':stay_end},ignore_index=True)
                
    return data

## Check added patient stay data for duplicates or error entries 

In [ ]:
# Call patient_stays function 
stay_data = patient_stays()

# Number of rows in data
print(len(stay_data.index))

# Check for duplicates (e.g., there should not be two entries with same date and room_number)
stay_data = stay_data.drop_duplicates(subset=['date','room_number'])

# Number of rows in clean data
print(len(stay_data.index))

## Create dataframe for each day and each hospital room

In [ ]:
# Define function that creates dataframe with entry for each day in desired date range and each hospital room ("room_table")
def room_table():
    cols = ['date','room_number']
    room_df = pd.DataFrame(columns = cols)
    for date in dates:
        for room in rooms:
            current_room = room
            current_date = date
            room_df = room_df.append({'date':current_date,'room_number':current_room},ignore_index=True)
    return room_df        

In [ ]:
# Call function
room_df = room_table()  

## Merge room_table with stay_data for database schema 

In [ ]:
# Use python sqlite3 to merge
conn = sqlite3.connect(':memory:')

# Send room_df and stay_data to sql
room_df.to_sql('rooms', conn, index=False)
stay_data.to_sql('stay_data', conn, index=False)

In [ ]:
# SQL code to merge the two tables
qry = '''
    SELECT a.date, a.room_number, b.patient_SSN, b.stay_start, b.stay_end
    FROM rooms a
        LEFT JOIN stay_data b ON (a.date >= b.stay_start AND a.date <= b.stay_end) AND a.room_number = b.room_number
    ORDER BY a.date, a.room_number;
    '''
# Send output of SQL query to pandas df
df = pd.read_sql_query(qry,conn)

# Output dataframes to csv for additional error checking
stay_data.to_csv('data/stay_data_check.csv')
df.to_csv('data/df_check.csv')

## Create dataset for stays and room_stays tables

In [ ]:
# Add new availability column (true=room available, false = room occupied)
df['room_available'] = np.where(np.isnan(df['patient_SSN']),True,False)

# Remove timestamp from date column
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date

# Convert patient_SSN from float64 to object
df['patient_SSNs'] = np.where(np.isnan(df['patient_SSN']),'Null',df['patient_SSN'])
df['patient_SSNs'] = df['patient_SSNs'].str.replace('.0','',regex=False)

In [ ]:
# Create SQL insert statements for 'room_stays' table
def room_stays():
    for index, row in df.iterrows():
        with open("outputs/room_stays.txt", "a+") as room_stays_text:
            print(f"INSERT INTO room_stays (room_number, room_date, patient_SSN, room_available) VALUES({row['room_number']},'{row['date']}',{row['patient_SSNs']},{row['room_available']});", file=room_stays_text)
    
room_stays()

In [ ]:
# Create SQL insert statements for 'stays' table
def stays():
    for index, row in stay_data.iterrows():
        with open("outputs/stays.txt","a+") as stays_text:
            print(f"INSERT INTO stays (patient_SSN, room_number, stay_start, stay_end) VALUES({row['patient_SSN']},{row['room_number']},'{row['stay_start']}','{row['stay_end']}');",file=stays_text)

stays()

## Create dataset for procedures table

In [ ]:
# Here, it is assumed that any patient that stayed in the hospital longer than 20 days had a procedure

# Calculate stay length
stay_data['los'] = stay_data['stay_end'] - stay_data['stay_start']
stay_data['los_days'] = stay_data['los'].astype('timedelta64[D]')

# Filter for patients who had a stay length > 10 days
stay_data_proc = stay_data[stay_data['los']>='20 days']

# Check for duplicate patients staying in hospital >20 days
dup_pat_check = stay_data_proc['patient_SSN'].duplicated().any()
print(dup_pat_check) # False = no dups

In [ ]:
def undergoes():
    surgeon_ids = [3,7,10,12,13]
    exam_rooms = ['A','B','C']
    for index, row in stay_data_proc.iterrows():
        patient = row['patient_SSN'] 
        procedure = random.randint(1,18)
        date = row['date']
        physician = random.choice(surgeon_ids)
        nurse = random.randint(1,3)
        exam_room = random.choice(exam_rooms)
        with open("outputs/undergoes.txt","a+") as proc_text:
            print(f"INSERT INTO undergoes (patient, procedure, surgery_date, physician, nurse) VALUES({patient},{procedure},'{date}',{physician},{nurse});",file=proc_text)
            
        with open("outputs/appointments.txt","a+") as appt_text:
            print(f"INSERT INTO appointments (patient_id, prep_nurse, physician, appointment_date, exam_room) VALUES({patient},{nurse},{physician},'{date}','{exam_room}');",file=appt_text)
            

undergoes()

## Create prescribes dataset + update appointments data

In [ ]:
def prescribes():
    exam_rooms = ['A','B','C']
    for index, row in stay_data_proc.iterrows():
        bool_test = random.randint(0,1)
        if (bool_test == 1):
            physician = random.randint(1,13)
            patient = row['patient_SSN']
            medication = random.randint(1,7)
            date = row['date']
            dose = random.choice([5,25,50,125,200])
            with open("outputs/prescribes.txt","a+") as pres_text:
                print(f"INSERT INTO prescribes (physician, patient, medication, appt_date, dose) VALUES({physician},{patient},{medication},'{appt_date}','{dose}');",file=pres_text)

prescribes()